In [ ]:
MODEL="unsloth/Llama-3.2-11B-Vision-Instruct-bnb-4bit"
CUDA=True
USER="user"
ASSISTANT="assistant"
START_HEADER="<|eot_id|><|start_header_id|>"
END_HEADER="<|end_header_id|>\n"
IMAGE="<|image|>"
def ROLE(str):
    return START_HEADER + str + END_HEADER
prompt = "<|begin_of_text|>"
image=[]

In [ ]:
from transformers import AutoProcessor, AutoModelForImageTextToText
from PIL import Image

In [ ]:
processor = AutoProcessor.from_pretrained(MODEL)
model = AutoModelForImageTextToText.from_pretrained(MODEL)

In [ ]:
def generate(msg, *args, **kwargs):
    global prompt
    global image

    prompt += ROLE(USER)
    for arg in args:
        prompt += IMAGE
        image.append(arg)
    prompt += msg
    prompt += ROLE(ASSISTANT)

    inputs = None
    if len(image) > 0:
        inputs = processor(text=prompt, images=image, return_tensors="pt")
    else:
        inputs = processor(text=prompt, return_tensors="pt")

    if CUDA:
        inputs = inputs.to("cuda")

    generatedids = model.generate(inputs.input_ids, max_length=256, do_sample=True) # remove do_sample
    output = processor.batch_decode(generatedids)[0]

    print(output)
    prompt += output[-1]

    return output[-1]

In [ ]:
print(generate("Hi"))